# Documentation for the creation and usage of the heatpump library (hplib)




In [ ]:
import hplib as hpl
import hplib_database as db
import pandas as pd
import matplotlib.pyplot as plt

## Defintions

**Abbreviation**

| Abbreviation | Meaning |
| --- | --- |
| P_th | Thermal Power in W |
| P_el | electical Power in W |
| COP | Coefficient of Performance |
| P_th_ref | Thermal Power in W |
| P_el_ref | electical Power in W |
| COP_ref | electical Power in W |


**Group IDs**
| Group ID | Type | Subtype |
| --- | --- | --- |
| 1 | Outdoor Air / Water | Inverter | 
| 2 | Brine / Water | Inverter | 
| 3 | Water / Water | Inverter | 
| 4 | Outdoor Air / Water | On-Off | 
| 5 | Brine / Water | On-Off | 
| 6 | Water / Water | On-Off | 
| 7 | Outdoor Air / Water | 2-Stages | 
| 8 | Brine / Water | 2-Stages |
| 9 | Water / Water | 2-Stages |

## Database Preparation
1. we downloaded all manufacturer data from https://keymark.eu/en/products/heatpumps/certified-products
2. then we unzipped the files and used the bash-skript to convert pdf into txt
3. afterwards we used this following functions to create and extent the heatpump keymark database

In [ ]:
# Import keymark data and save to csv database
db.ImportKeymarkData()
# -> this creates /output/database_keymark.csv)

In [ ]:
# Reduce to climate measurement series with average climate, delete redundant entries and save to csv sub-database
db.ReduceKeymarkData('database_keymark.csv','average')
# -> this creates /output/database_keymark_average.csv)

In [ ]:
# Normalize electrical and thermal power from the keymark database to values from setpoint T_in = -7 °C and T_out = 52 °C
db.NormalizeKeymarkData('database_keymark_average.csv')
# -> this creates /output/database_keymark_average_normalized.csv)

In [ ]:
# Identify subtypes like On-Off, Inverter and 2-stages Heatpump and assign group depending on its type and subtype
db.IdentifySubtypes('database_keymark_average_normalized.csv')
# -> this creates /output/database_keymark_average_normalized_subtypes.csv)

In [ ]:
#Calculate parameters p1-p3 for P_th, P_el and COP
db.CalculateFunctionParameters('database_keymark_average_normalized_subtypes.csv')
# -> this creates the final hplib_database.csv

In [ ]:
#Calculate generic heatpump models for each group
db.addGeneric()
# -> this overwrites the hplib_database.csv

In [2]:
#Reduce to unique Values
db.ReduceToUnique()
# -> this overwrites the hplib_database.csv

## Usage for simulation

In [ ]:
# Have a look into the hplib database
database = hpl.loadDatabase()
database

In [ ]:
#Show how many unique heatpumps are in each group
Groups=[1,2,3,4,5,6,7,8,9]
for group in Groups:
    dbgroup=database.loc[(database['Group']==group)&(database['Model']!='Generic'),]
    print(group,len(pd.unique(dbgroup['p3_P_el [-]']).tolist()),len(dbgroup['p1_P_el [1/°C]'].tolist()))

In [ ]:
# Define a specific heatpump model and get parameters
parameters = hpl.getParameters('i-SHWAK V4 06')
parameters

In [ ]:
# Alternatively, define a "Generic" heatpump model and get parameters
# For every Group ID there is a parameter set which represents an average heatpump of its group
# Please define the Type "Generic", the Group ID (1-9) and P_th_ref in W
parameters = hpl.getParameters('Generic',1,10000)
parameters

In [ ]:
# Simulate on time step
# Please define T_in_primary, T_in_secondary in °C and the parameters from the previous step 
# The T_in_secondary is supposed to be heated up by 5 K
P_th, P_el, COP = hpl.simulate(-7, 47, parameters)
print('P_th: '+str(int(P_th))+' W')
print('P_el: '+str(int(P_el))+' W')
print('COP: '+str(round(COP,3)))

In [ ]:
# For a time series create a loop with the function

## Overview of the efficiency range of the heatpump models within the database

In [ ]:
# Plotting the COPs for every heatpump, sorted by Group ID

para = pd.read_csv('hplib_database.csv')
#Plot Pth, Pel, COP for choosen models
Groups=[1,2,4,5] #define wich groups to plot
for groups_c in Groups:
    Group1=para.loc[para['Group']==groups_c,:] 
    T_in=[*range(-15,36,1)]#inflow TEMP
    T_out=[30,55]
    #get parameters
    Models=(Group1['Model'].values.tolist())
    p1_P_th=(Group1['p1_P_th [1/°C]'].values.tolist())
    p2_P_th=(Group1['p2_P_th [1/°C]'].values.tolist())
    p3_P_th=(Group1['p3_P_th [-]'].values.tolist())
    p1_P_el=(Group1['p1_P_el [1/°C]'].values.tolist())
    p2_P_el=(Group1['p2_P_el [1/°C]'].values.tolist())
    p3_P_el=(Group1['p3_P_el [-]'].values.tolist())
    p1_COP=(Group1['p1_COP [-]'].values.tolist())
    p2_COP=(Group1['p2_COP [-]'].values.tolist())
    p3_COP=(Group1['p3_COP [-]'].values.tolist())
    p_el_ref=Group1['P_el_ref [W]'].values.tolist()
    p_th_ref=Group1['P_th_ref [W]'].values.tolist()
    i=0 #iterator for-loop
    #create Dataframes to plot
    COP_low_g=pd.DataFrame()
    COP_low_g['T_in']=T_in
    COP_high_g=pd.DataFrame()
    COP_high_g['T_in']=T_in
    Average_g=pd.DataFrame()
    Average_g['T_in']=T_in
    #get average values
    k7_average=Group1[-1:]['p1_COP [-]'].values.tolist()[0]
    k8_average=Group1[-1:]['p2_COP [-]'].values.tolist()[0]
    k9_average=Group1[-1:]['p3_COP [-]'].values.tolist()[0]
    Group_average=Group1[-1:]
    for number in p1_P_th: #iterate by all model
        COP_low=[]
        COP_high=[]
        Average_cop_low=[]
        Average_cop_high=[]
        for tin in T_in: #calculate Thermal and electric Power and COP
            p_th_low,p_el_low,cop_low=hpl.simulate(tin,T_out[0],Group1.iloc[i:i+1])
            p_th_high,p_el_high,cop_high=hpl.simulate(tin,T_out[1],Group1.iloc[i:i+1])
            COP_low.append(cop_low)
            COP_high.append(cop_high)
            average_pthlow,average_pellow,average_low=hpl.simulate(tin,T_out[0],Group_average)
            average_pthhigh,average_pelhigh,average_high=hpl.simulate(tin,T_out[1],Group_average)
            #calculate for average values
            Average_cop_low.append(average_low)
            Average_cop_high.append(average_high)
        #write values in a dataframe
        COP_low_g[Models[i]]=COP_low
        COP_high_g[Models[i]]=COP_high
        Average_g['COP_low']=Average_cop_low
        Average_g['COP_high']=Average_cop_high
        i=i+1 #increase iterator


    #Plot: COP
    fig, axes = plt.subplots(nrows=1, ncols=1)
    variable=str(groups_c)
    COP_low_g.plot(ax=axes, x='T_in', y=Models, kind='line', c='Blue', title='Group:' + variable + '\n COP', legend=False)
    COP_high_g.plot(ax=axes, x='T_in', y=Models, kind='line', c='Red', legend=False)
    Average_g.plot(ax=axes, x='T_in', y='COP_low', kind='line', c='Black', legend=False)
    Average_g.plot(ax=axes, x='T_in', y='COP_high', kind='line', c='Black', legend=False)
    plt.show()
